# CS295/395: Secure Distributed Computation
## In-Class Exercise, week of 10/03/2022

In [8]:
# Imports and definitions
import numpy as np
from collections import defaultdict
import numpy as np
import galois

from nacl.public import PrivateKey, Box, SealedBox

# GF = galois.GF(2**13 - 1)
GF_2 = galois.GF(2) # we work in the binary field this week!

# Library for circuits
from dataclasses import dataclass

@dataclass
class Gate:
    type: str
    in1: int
    in2: int
    out: int

@dataclass
class Circuit:
    inputs: any
    outputs: any
    gates: any
        
def print_circuit(c):
    print('inputs:', c.inputs)
    print('outputs:', c.outputs)
    print('gates:')
    for g in c.gates:
        print('  ', g)

## Party Class

In [9]:
class Party:
    """A participant in a multiparty computation protocol."""
    def __init__(self):
        """Initialize the field size and dictionary to hold received messages."""
        self.input = None
        self.output = None
        self.received = defaultdict(list)
    
    def send(self, other, round, msg):
        """Simulate sending a message `msg` to another party `other` during round `round`"""
        other.received[round].append(msg)

    def get_view(self):
        """Returns the view of this party: its input, output, and received messages."""
        return (self.input, self.output, dict(self.received))

## Question 1

Describe the 1-out-of-2 *oblivious transfer* (OT) protocol. Reference Section 3.7 in Pragmatic MPC.

**Idea**:
- R generates two keypairs, but throws away one of the secret keys
- S encrypts both secrets using the two public keys from R
- R can decrypt only one of the secrets, because they retained only one secret key

**Setup**:
- S knows two secrets x1 and x2
- R knows a selection bit b

**Protocol**:
- **Round 1**: R generates two keypairs: sk1, pk1 and sk2, pk2. R throws away sk2.
    - If b = 0, R sends (pk1, pk2) to S
    - If b = 1, R sends (pk2, pk1) to S
- **Round 2**: S receive (pka, pkb). S sends (Enc_pka(x1), Enc_pkb(x2)) to R
- **Round 3**: R receive (c1, c2)
    - If b = 0, R decrypts c1 using sk1 (to recover x1)
    - If b = 1, R decrypts c2 using sk1 (to recover x2)

## Question 2

Why is the oblivious transfer protocol secure against semi-honest adversaries? Why is it not secure against malicious adversaries?

- It is secure against semi-honest adversaries because R will always correctly throw away sk2. 
- A malicious R could keep sk1 and sk2, and decrypt everything in Round 3

## Question 3

Implement 1-out-of-2 OT.

In [10]:
class OT_Sender(Party):
    # x1 and x2 are the secrets
    def round1(self, x1, x2, receiver):
        self.x1 = x1
        self.x2 = x2
        self.receiver = receiver

    def round2(self):
        # Round 2: S receive (pka, pkb). S sends (Enc_pka(x1), Enc_pkb(x2)) to R
        [(pka, pkb)] = self.received[1]
        x1_b = int(self.x1).to_bytes(1, 'little')
        x2_b = int(self.x2).to_bytes(1, 'little')
        
        x1_enc = SealedBox(pka).encrypt(x1_b)
        x2_enc = SealedBox(pkb).encrypt(x2_b)
        
        self.send(self.receiver, 2, (x1_enc, x2_enc))
        
    
    def round3(self):
        pass

class OT_Receiver(Party):
    def round1(self, b, sender):
        self.sender = sender
        self.b = b
        
        # Round 1: R generates two keypairs: sk1, pk1 and sk2, pk2. R throws away sk2.
        keypair1 = PrivateKey.generate() # keep this one
        keypair2 = PrivateKey.generate() # throw this one away after this round
        
        self.saved_key = keypair1
        
        # If b = 0, R sends (pk1, pk2) to S
        # If b = 1, R sends (pk2, pk1) to S
        if self.b == 0:
            self.send(self.sender, 1, (keypair1.public_key, keypair2.public_key))
        elif self.b == 1:
            self.send(self.sender, 1, (keypair2.public_key, keypair1.public_key))
        
    
    def round2(self):
        pass
    
    def round3(self):
        # Round 3: R receive (c1, c2)
        # If b = 0, R decrypts c1 using sk1 (to recover x1)
        # If b = 1, R decrypts c2 using sk1 (to recover x2)
        [(c1, c2)] = self.received[2]
        
        if self.b == 0:
            plaintext = SealedBox(self.saved_key).decrypt(c1)
        elif self.b == 1:
            plaintext = SealedBox(self.saved_key).decrypt(c2)
        
        self.output = int.from_bytes(plaintext, 'little')
        return self.output

In [11]:
# TEST CASE
sender = OT_Sender()
receiver = OT_Receiver()

# Round 1
sender.round1(GF_2(0), GF_2(1), receiver)
receiver.round1(GF_2(1), sender)

# Round 2
sender.round2()
receiver.round2()

# Round 3
sender.round3()
output = receiver.round3()

print("Receiver's output:", output)
assert output == 1

Receiver's output: 1


## Question 4

Describe 1-out-of-4 OT.

- In 1-out-of-2 OT, the sender has 2 secrets, and the receiver receives 1 of them
- In 1-out-of-4 OT, the sender has 4 secrets, and the receiver receives 1 of them

**Protocol**:
- **Round 1**: R generates four keypairs: sk1, pk1, sk2, pk2, sk3, pk3, sk4, pk4. R keeps on;y sk1 (throwing away 3 sks)
    - If b1 = 0 and b2 = 0, R sends (pk1, _, _, _) to S
    - If b1 = 0 and b2 = 1, R sends (_, pk1, _, _) to S
    - If b1 = 1 and b2 = 0, R sends (_, _, pk1, _) to S
    - If b1 = 1 and b2 = 1, R sends (_, _, _, pk1) to S
- **Round 2**: 
    - S receive (pka, pkb, pkc, pkd).
    - S sends (Enc_pka(x1), Enc_pkb(x2), Enc_pkc(x3), Enc_pkd(x4)) to R
- **Round 3**: R receive (c1, c2, c3, c4)
    - If b1 = 0 and b2 = 0, R decrypts c1 using sk1 (to recover x1)
    - If b1 = 0 and b2 = 1, R decrypts c2 using sk1 (to recover x2)
    - If b1 = 1 and b2 = 0, R decrypts c3 using sk1 (to recover x3)
    - If b1 = 1 and b2 = 1, R decrypts c4 using sk1 (to recover x4)

## Question 5

Describe a method for evaluating an `AND` gate using 1-out-of-4 OT on additive-secret-shared inputs.

- P1 and P2 each hold one additive share of the two input wire values
- We use 1 out of 4 OT with P1 as S and P2 as R
- P1 will generate their output share randomly
- We will build a truth table for P2's output share
- P1 can say
  - Given my input shares and my random output shares
  - What would p2's output share be for each possible value of P2's input shares
  - We build a table of these, and use the potential output shares for P2 as the OT secrets
  - P2 uses its shares as the OT selection bits


**Protocol**:
- Inputs: P1 has s1_i, s1_j; P2 has s2_i, s2_j (additive shares of the AND gates input)


**Round1** P2 generates keypaires and keep one of them
- P2 generates 4 keypairs
- P2 keeps one secret key based on the values of s2_i and s2_j
  - if s2_i = 0 and s2_j = 0, R sends (pk1, _, _, _) to S
  - ...


**Round2** P1 generates the truth table as its secrets and encrypts the values of it
- P1 recieves 4 public keys from P2
- P1 generates a random output share r = s1_k
- P1 calls T_G to get the truth table, using s1_i, s1_j, and r as inputs
- P1 encrypts each row of the truth table using the 4 public keys


**Round3** P2 decrypts the row of the truth table corresponding to its actual shares
- P2 decrypts the right row of the trith table using sk1
  
At the end, P1 has s1_k, P2 has s2_k and s1_k + s2_k = output of the gate



## Question 6

Implement the function $T_G$ that computes the truth table for an `AND` gate with input wires $i$ and $j$ based on input shares of P1 and P2 and output share $r$ for P1.

In [12]:
# P1 holds shares s1_*
# P2 holds shares s2_*
def S(s1_i, s1_j, s2_i, s2_j):
    return (s1_i + s2_i) * (s1_j + s2_j)

# P1 holds shares s1_*
# P1 generates a random output share r
def T_G(r, s1_i, s1_j):
    
    combinations = GF_2([(0,0), (0,1), (1,0), (1,1)])
    output_table = []
    for s2_i, s2_j in combinations:
        s2_k = r + S(s1_i, s1_j, s2_i, s2_j)
        output_table.append(s2_k)

    return output_table

T_G(GF_2(1), GF_2(0), GF_2(1))

[GF(1, order=2), GF(1, order=2), GF(0, order=2), GF(1, order=2)]

In [13]:
# TEST CASE
s1_i, s1_j, s2_i, s2_j = GF_2([0, 1, 1, 0])
row_num = 2 # because of the position of (1, 0) in the table computed by T_G
for _ in range(10): # try it 10 times, to account for randomness
    r = GF_2.Random()
    table = T_G(r, s1_i, s1_j)
    p2_share = table[row_num]
    output_result = r + p2_share
    assert output_result == GF_2(1)

## Question 7

Implement the `AND` gate protocol from above.

In [14]:
class AND_P1(Party):
    # x1 and x2 are the secrets
    def round1(self, s1_i, s1_j, p2):
        self.s1_i = s1_i
        self.s1_j = s1_j
        self.p2 = p2

    def round2(self):
        # Round 2: S receive (pka, pkb). S sends (Enc_pka(x1), Enc_pkb(x2)) to R
        [pks] = self.received[1]
       
#         P1 generates a random output share r = s1_k
#         P1 calls T_G to get the truth table, using s1_i, s1_j, and r as inputs
        r = GF_2.Random()
        self.output = r
        truth_table = T_G(r, self.s1_i, self.s1_j)
        encrypted_truth_table = []
        for pk, table_element in zip(pks, truth_table):
            table_element_b = int(table_element).to_bytes(1, 'little')
            enc = SealedBox(pk).encrypt(table_element_b)
            encrypted_truth_table.append(enc)
       
        self.send(self.p2, 2, encrypted_truth_table)
   
    def round3(self):
        return self.output

class AND_P2(Party):
    def round1(self, s2_i, s2_j, p1):
        self.p1 = p1
        self.s2_i = s2_i
        self.s2_j = s2_j
#         P2 generates 4 keypairs
#         P2 keeps one secret key, based on the values of s2_i, s2_j
        # Round 1: R generates two keypairs: sk1, pk1 and sk2, pk2. R throws away sk2.
        keypair1 = PrivateKey.generate() # keep this one
        keypair2 = PrivateKey.generate() # throw this one away after this round
        keypair3 = PrivateKey.generate() # throw this one away after this round
        keypair4 = PrivateKey.generate() # throw this one away after this round

        self.saved_key = keypair1
       
        if s2_i == 0 and s2_j == 0:
            self.send(self.p1, 1, (keypair1.public_key,
                                   keypair2.public_key,
                                   keypair3.public_key,
                                   keypair4.public_key))
        elif s2_i == 0 and s2_j == 1:
            self.send(self.p1, 1, (keypair2.public_key,
                                   keypair1.public_key,
                                   keypair3.public_key,
                                   keypair4.public_key))
        elif s2_i == 1 and s2_j == 0:
            self.send(self.p1, 1, (keypair3.public_key,
                                   keypair2.public_key,
                                   keypair1.public_key,
                                   keypair4.public_key))
        elif s2_i == 1 and s2_j == 1:
            self.send(self.p1, 1, (keypair4.public_key,
                                   keypair2.public_key,
                                   keypair3.public_key,
                                   keypair1.public_key))

   
    def round2(self):
        pass
   
    def round3(self):
        [(c1, c2, c3, c4)] = self.received[2]
       
        if self.s2_i == 0 and self.s2_j == 0:
            plaintext = SealedBox(self.saved_key).decrypt(c1)
        elif self.s2_i == 0 and self.s2_j == 1:
            plaintext = SealedBox(self.saved_key).decrypt(c2)
        elif self.s2_i == 1 and self.s2_j == 0:
            plaintext = SealedBox(self.saved_key).decrypt(c3)
        elif self.s2_i == 1 and self.s2_j == 1:
            plaintext = SealedBox(self.saved_key).decrypt(c4)
       
        self.output = GF_2(int.from_bytes(plaintext, 'little'))
        return self.output

In [15]:
# TEST CASE

for _ in range(5): # try it a few times
    p1 = AND_P1()
    p2 = AND_P2()
    s1_i, s1_j, s2_i, s2_j = GF_2([0, 1, 1, 0])

    # Round 1
    p1.round1(s1_i, s1_j, p2)
    p2.round1(s2_i, s2_j, p1)

    # Round 2
    p1.round2()
    p2.round2()

    # Round 3
    output_share1 = p1.round3()
    output_share2 = p2.round3()

    print("P1's output:", output_share1)
    print("P2's output:", output_share2)
    assert output_share1 + output_share2 == 1

P1's output: 1
P2's output: 0
P1's output: 0
P2's output: 1
P1's output: 1
P2's output: 0
P1's output: 1
P2's output: 0
P1's output: 1
P2's output: 0


## Question 8

Describe the GMW protocol for evaluating a binary circuit.

* Refer to 10-03-22 Q3 to write GMW *

**GMW**
- Additive Shares
- Eval Gates
  - AND gate uses OT
- Reconstruct outputs


**Inputs**
- P1 has (binary) values for some of the input wires
- P2 has (binary) values for some of the input wires

**Protocol**
- Input section
  - **Round 1** (phase 1): Each party secret shares its input bits to the other party (same as BGW but with additive) and adds its own shares to the wire_vals dictionary (create two shares and put one in dict and send other to other party)
  - **Round 2** each party recieves shares of the other parties inputs and adds these to the wire_vals dictionary


- Eval section
  - ** R 2 < i < k** Eval next gate $g$ in circuit
    - XOR: `wire_vals[g.out] = wire_vals[g.in1] + wire_vals[g.in2]` (similar to addition gate in BGW)
    - INV (NOT): `wire_vals[g.out] = wire_vals[g.in1] + GF_2(1)`
      - `g.in2 = -1` in this case to indicate that its not used
    - AND: hard case - use the protocol from Q7
      - inputs: wire_vals[g.in1] and wire_val[g.in2]
      - outputs: one secret share for g.out
      - implement 3 ot-phases
        - OTP1: generate pub keys (r1 from Q7)
        - OTP2: generate and encrypt truth table (r2)
        - OPT3: decrypt one row from the TT (r3)
- Output section
  - ** round k ** parties broadcast their shares of the output wire values
  - ** round k+1** parties reconstruct the output wire values using their own shares plus the broadcasted shares recieved from the other party

## Question 9

Describe the main idea behind Yao's garbled circuit protocol.

YOUR ANSWER HERE

## Question 10

What are the primary advantages and disadvantages of Yao's garbled circuit protocol?

YOUR ANSWER HERE